# Imports

In [34]:
#libraries for cleaning text 
import nltk
import string
import math
import numpy as np
import pandas as pd
import contractions
from bs4 import BeautifulSoup

#libraries to create decision tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import pickle

#librari to read and write files
import csv

#miscelanious
import os

# Load dataset

In [35]:
all_reviews_df = pd.read_csv('./data/imdb_dataset.csv',encoding='ISO-8859-1')

# Cleaning and transforming data

In [36]:
all_reviews_df['Label'] = all_reviews_df['Label'].map(lambda label: 1 if label == "pos" else 0 )
all_reviews_df.head()

,Review,Label
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0


In [37]:
reviews_df = all_reviews_df.loc[10000:20000]  #muestra de 10001
reviews_df.head()

,Review,Label
10000,"This movie is a bizarre fantasy tale, that I'm...",0
10001,Albert Pyun presents his vision of the lost ci...,0
10002,"I bought the DVD to get Julia Ormond. Well, I ...",0
10003,"This story of Ted Brice, an American pilot who...",0
10004,This started out as a good sketch comedy. The ...,0


In [64]:
reviews_df['Label'][10302]

0

In [38]:
def normalize_document(text):
    stop_words = set( nltk.corpus.stopwords.words('english')+ list(string.punctuation)+["...","*","''","``"])
    text_without_html = BeautifulSoup(text).get_text()
    words = text_without_html.split() 
    words_without_contractions = [contractions.fix(word) for word in words]
    #Join wordlist again to use word tokenize so words can be separated properly without losing meaning
    text_complete = ' '.join(words_without_contractions)
    words_nltk = nltk.word_tokenize(text_complete)
    clean_words = [word.lower() for word in words_nltk if word.lower() not in stop_words]
    clean_text = " ".join(clean_words)
    return clean_text

In [39]:
def normalize_corpus(reviews):
    return np.array([normalize_document(review)for review in reviews])  

In [40]:
reviews_list = reviews_df['Review'].to_list()
normalized_corpus = normalize_corpus(reviews_list)

#  Count Vectorizer

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

In [42]:
cv = CountVectorizer() #Parametros con valores por defecto
cv_matrix = cv.fit_transform(normalized_corpus) #combina fit y luego trasform
cv_matrix

<10001x50881 sparse matrix of type '<class 'numpy.int64'>'
	with 998467 stored elements in Compressed Sparse Row format>

In [43]:
df = pd.DataFrame(cv_matrix.toarray(), columns=cv.get_feature_names())
df.head()

,00,000,00000000000,007,0079,0083,009,00am,00o,00schneider,...,zzzzzzzzz,zã,zã¼mcã¼,â½,â¾,ã³r,ã¼ber,ã¼bermensch,ã½,ã½s
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
#agregando label al dataframe
df['Label'] = reviews_df['Label'].to_list()
df.head()

,00,000,00000000000,007,0079,0083,009,00am,00o,00schneider,...,zã,zã¼mcã¼,â½,â¾,ã³r,ã¼ber,ã¼bermensch,ã½,ã½s,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training model: Decision Tree

In [45]:
feature_cols = cv.get_feature_names() 
X = df[feature_cols] 
y = df["Label"] 

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 
# 70% training, 30% test

In [47]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(7000, 50881) (3001, 50881) (7000,) (3001,)


In [48]:
review_tree_classifier = RandomForestClassifier(n_estimators=50)

In [49]:
review_tree_classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [50]:
print('accuracy del clasificador - version 3 : {0:.2f}'.format(accuracy_score(y_test, review_tree_classifier.predict(X_test))))
# confusion matrix
print('matriz de confusión del clasificador - version 3: \n {0}'.format(confusion_matrix(y_test, review_tree_classifier.predict(X_test))))
# precision 
print('precision del clasificador - version 3 : {0:.2f}'.format(precision_score(y_test, review_tree_classifier.predict(X_test))))
# precision 
print('recall del clasificador - version 3 : {0:.2f}'.format(recall_score(y_test, review_tree_classifier.predict(X_test))))
# f1
print('f1 del clasificador - version 3 : {0:.2f}'.format(f1_score(y_test, review_tree_classifier.predict(X_test))))

accuracy del clasificador - version 3 : 0.82
matriz de confusión del clasificador - version 3: 
 [[ 243  519]
 [  23 2216]]
precision del clasificador - version 3 : 0.81
recall del clasificador - version 3 : 0.99
f1 del clasificador - version 3 : 0.89


# Save vectorizer 

In [51]:
vectorizer_path = os.path.join("Vectorizer", "countVectorizer.pkl")

In [52]:
vectorizer_file = open(vectorizer_path,"wb")

In [53]:
pickle.dump(cv,vectorizer_file)

In [54]:
vectorizer_file.close()

# Save classifier

In [55]:
classifier_path = os.path.join("classifier", "reviewClasiffierTree.pkl")

In [56]:
classifier_file = open(classifier_path, "wb")

In [57]:
pickle.dump(review_tree_classifier, classifier_file)

In [58]:
classifier_file.close()